In [7]:
from ditopic import MOF_ditopic
import numpy as np

template = 'fcu.cif' #UIO6x #lib file 
node_connection= 12   #lib file 

templates_dir = 'template_database' #default 
nodes_dir = 'nodes' #default 
edges_dir = 'edges' #default 
new_mof = MOF_ditopic(templates_dir,nodes_dir,edges_dir,template,node_connection)
new_mof.load(False)


template : fcu.cif

Number of vertices =  4
Number of edges =  24

*****************************************************************
RMSD of the compatible node BBs with assigned vertices:          
*****************************************************************

vertex V (12 connected)
     12c_Zr_1_Ch.cif deviation = 0.0 (within tolerance)
* 1 compatible building blocks out of 1 available for node V *

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
vertex assignment :  ['v1-12c_Zr_1_Ch.cif']
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
edge assignment :  ('2B_2F_Ch.cif',)
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

scaling unit cell and vertex positions...
optimizing with local minimization algorithm L-BFGS-B...

The final objective function value

In [8]:
supercell= np.asarray([0,0,0])
new_mof.basic_supercell(supercell)
new_mof.write_basic_supercell('21.gro','22.xyz')
new_mof.defect_missing()
new_mof.term_defective_model()
new_mof.write_tntemof('23.gro')

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

this MOF has 1 fragment
UNsaturated edges(linkers) exist, need linker_termination <= 15
UNsaturated nodes exist, <=13 nodes need node_termination
degree of edge E1 is 1, neighbor nodes are [3]
degree of edge E4 is 1, neighbor nodes are [3]
degree of edge E5 is 1, neighbor nodes are [2]
degree of edge E6 is 1, neighbor nodes are [2]
degree of edge E9 is 1, neighbor nodes are [2]
degree of edge E12 is 1, neighbor nodes are [2]
degree of edge E13 is 1, neighbor nodes are [1]
degree of edge E14 is 1, neighbor nodes are [1]
degree of edge E15 is 1, neighbor nodes are [1]
degree of edge E17 is 1, neighbor nodes are [1]
degree of edge E18 is 1, neighbor nodes are [1]
degree of edge E20 is 1, neighbor nodes are [1]
degree of edge E21 is 1, neighbor nodes are [1]
degree of edge E22 is 1, neighbor nodes are [1]
degree of edge E23 is 1, neighbor nodes are [1]
1 4
4 4
5 4
6 4
9 4
12 4
13 8
14 4
15 4
17 4
18 4
20 4
21 4
22 4
23 4


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [9]:
#scalar parameter to adjust boundary 
new_mof.basic_supercell(supercell,scalar=0,boundary_scalar=-0)
new_mof.write_basic_supercell('24.gro','25.xyz')

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [10]:
new_mof.unsaturated_main_frag_edges,new_mof.unsaturated_main_frag_nodes

([('E1', 1),
  ('E4', 1),
  ('E5', 1),
  ('E6', 1),
  ('E9', 1),
  ('E12', 1),
  ('E13', 1),
  ('E14', 1),
  ('E15', 1),
  ('E17', 1),
  ('E18', 1),
  ('E20', 1),
  ('E21', 1),
  ('E22', 1),
  ('E23', 1)],
 [(3, 10),
  (14, 5),
  (4, 8),
  (2, 11),
  (13, 5),
  (12, 8),
  (11, 3),
  (9, 2),
  (7, 2),
  (5, 2),
  (6, 2),
  (10, 2),
  (8, 1)])

In [11]:
new_mof.defect_missing([],[1,4,5,6,9,12,13,14,15,17,18,20,21,22,23]) #1,4,5,6,9,12,13,14,15,17,18,20,21,22,23
new_mof.term_defective_model()
new_mof.write_tntemof('26.gro')

this MOF has 1 fragment
only saturated edges(linkers) exist
UNsaturated nodes exist, <=14 nodes need node_termination


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [14]:
#sub
sub_file = 'Di_SUB.pdb'
sub_class = 'EDGE'
candidate_res_idx_list = [2,3,7,8]
sub_res_newname = 'SUB'
new_mof.defect_replace_linker(sub_file,sub_class,candidate_res_idx_list)
new_mof.write_view_replaced('35.gro')

3Dmol.js failed to load for some reason. Please check your browser console for error messages.